<a href="https://colab.research.google.com/github/AnugyaMaurya/AnugyaMaurya/blob/main/ML_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [2]:
# Step 1: Upload the Zip File
from google.colab import files

# Upload the zip file
uploaded = files.upload()

# Step 2: Extract the Zip File
import zipfile
import os

# Get the name of the uploaded file
zip_file_name = list(uploaded.keys())[0]

# Define the directory where you want to extract the files
extract_dir = 'extracted_files'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the files in the extracted directory
extracted_files = os.listdir(extract_dir)
print('Extracted files:', extracted_files)


Saving archive (2).zip to archive (2) (1).zip
Extracted files: ['val', 'train']


In [3]:
import os

# List all files in the extracted directory
extracted_files = os.listdir(extract_dir)
print('Extracted files:', extracted_files)


Extracted files: ['val', 'train']


In [4]:
import os
import pandas as pd

# Define the directories
train_dir = os.path.join('extracted_files', 'train')
val_dir = os.path.join('extracted_files', 'val')

# List files in the train and val directories
train_files = os.listdir(train_dir)
val_files = os.listdir(val_dir)

print('Train files:', train_files)
print('Validation files:', val_files)

# Define the path to the Excel file (assuming it's in the 'train' directory)
excel_file_name = 'train.xlsx'  # Replace with the actual file name if different
excel_file_path = os.path.join(train_dir, excel_file_name)

# Check if the file exists
if os.path.exists(excel_file_path):
    # Load the Excel file into a DataFrame
    df = pd.read_excel(excel_file_path)

    # Display the first few rows of the DataFrame
    print(df.head())
else:
    print(f"File {excel_file_name} not found in {train_dir}")


Train files: ['img', 'label']
Validation files: ['img', 'label']
File train.xlsx not found in extracted_files/train


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import numpy as np
from PIL import Image

# Define paths
train_image_dir = os.path.join('extracted_files', 'train', 'img')
train_label_dir = os.path.join('extracted_files', 'train', 'label')
val_image_dir = os.path.join('extracted_files', 'val', 'img')
val_label_dir = os.path.join('extracted_files', 'val', 'label')

# Function to load and preprocess images and labels in batches
def load_images_and_labels_in_batches(image_dir, label_dir, batch_size=32):
    images_batches = []
    labels_batches = []

    image_files = sorted(os.listdir(image_dir))
    label_files = sorted(os.listdir(label_dir))
    num_batches = len(image_files) // batch_size

    for i in range(num_batches):
        images_batch = []
        labels_batch = []
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        for img_file, lbl_file in zip(image_files[start_idx:end_idx], label_files[start_idx:end_idx]):
            img_path = os.path.join(image_dir, img_file)
            lbl_path = os.path.join(label_dir, lbl_file)

            # Read and resize the images
            image = Image.open(img_path).convert('RGB')
            image = image.resize((256, 256))
            images_batch.append(np.array(image))

            # Read and resize the labels (assuming they are single-channel grayscale)
            label = Image.open(lbl_path).convert('L')
            label = label.resize((256, 256))
            labels_batch.append(np.array(label))

        images_batches.append(np.array(images_batch))
        labels_batches.append(np.array(labels_batch))

    return images_batches, labels_batches

train_images_batches, train_labels_batches = load_images_and_labels_in_batches(train_image_dir, train_label_dir)
val_images_batches, val_labels_batches = load_images_and_labels_in_batches(val_image_dir, val_label_dir)

# Normalize images and labels in batches
for i in range(len(train_images_batches)):
    train_images_batches[i] = train_images_batches[i] / 255.0

for i in range(len(val_images_batches)):
    val_images_batches[i] = val_images_batches[i] / 255.0

# Ensure labels are binary (0 or 1) in batches
for i in range(len(train_labels_batches)):
    train_labels_batches[i] = (train_labels_batches[i] > 0).astype(np.uint8)

for i in range(len(val_labels_batches)):
    val_labels_batches[i] = (val_labels_batches[i] > 0).astype(np.uint8)

# Create data generators
train_generator = zip(train_images_batches, train_labels_batches)
val_generator = zip(val_images_batches, val_labels_batches)

# Create callbacks
checkpoint = ModelCheckpoint('road_segmentation_model.h5', save_best_only=True)

# Train the model using generators
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=20,
                    steps_per_epoch=len(train_images_batches),
                    validation_steps=len(val_images_batches),
                    callbacks=[checkpoint])


Epoch 1/20
92/92 [==============================] - ETA: 0s - loss: 0.0785 - accuracy: 0.9853

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


92/92 [==============================] - 229s 2s/step - loss: 0.0785 - accuracy: 0.9853 - val_loss: 0.0798 - val_accuracy: 0.9948
Epoch 2/20


92/92 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [8]:
model.save('road_segmentation_model.h5')

In [9]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('road_segmentation_model.tflite', 'wb') as f:
    f.write(tflite_model)


In [10]:
import cv2

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='road_segmentation_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess frame
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (256, 256))
    frame_normalized = frame_resized / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0).astype(np.float32)
    return frame_expanded

# Function to postprocess the output
def postprocess_output(output):
    output = output.squeeze()
    output_resized = cv2.resize(output, (frame.shape[1], frame.shape[0]))
    output_binary = (output_resized > 0.5).astype(np.uint8) * 255
    return output_binary

# Open video capture (0 for webcam, or provide video file path)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_data = preprocess_frame(frame)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    mask = postprocess_output(output_data)
    mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 0.7, mask_colored, 0.3, 0)

    cv2.imshow('Road Segmentation', overlay)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
